In [1]:
%reload_ext autoreload
%autoreload 2
#show all columnsdf
#ignore warnings.

In [8]:
import pandas as pd
pd.set_option('display.max_columns', 9000)
import numpy as np
import os, sys
import importlib
import yaml, json
from pathlib import Path
base=Path('./Data/NSF-Downloads-8july2024/')

In [ ]:
import pandas as pd
import zipfile
from pathlib import Path

def process_zip_files(base_path, output_path, min_years):
    """
    Processes ZIP files in the given base directory, extracts XML files, and returns a DataFrame
    with information about the extracted files for years > min_years.
    
    Parameters:
        base_path (Path): The base directory containing ZIP files.
        output_path (Path): The directory where files will be temporarily extracted.
        min_years (int): Minimum year to include in the results.
    
    Returns:
        pd.DataFrame: A DataFrame with columns ['file', 'year', 'xmlfiles'].
    """
    base = Path(base_path)
    output = Path(output_path)
    output.mkdir(parents=True, exist_ok=True)  # Ensure output directory exists
    
    zipfiles = list(base.rglob("*.zip"))  # List all ZIP files in base directory
    details = []
    
    for file in zipfiles:
        file_info = {}
        file_info['file'] = str(file)
        
        # Extract year from filename (assuming the year is in the filename)
        try:
            year = int(file.stem.split(".")[0])  # Adjust split logic based on actual filename format
            file_info['year'] = int(year)
        except ValueError:
            print(f"Could not extract year from filename: {file}")
            continue
        
        # Skip files with years <= min_years
        if year <= min_years:
            continue
        
        # Create a year-specific directory for extraction
        year_dir = output / str(year)
        year_dir.mkdir(parents=True, exist_ok=True)
        
        # Extract ZIP file contents to the year-specific directory
        try:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(year_dir)
            file_info['errors']=True
        except Exception as e:
            print(f"Failed to extract {file}: {e}")
            file_info['errors']=False
            continue
        
        # Count the number of XML files in the year-specific directory
        xmlfiles = list(year_dir.rglob("*.xml"))
        file_info['xmlfiles'] = len(xmlfiles)
        
        # Append file info to the details list
        details.append(file_info)
    
    # Convert the details list into a DataFrame
    df = pd.DataFrame(details)
    #sort by year decending
    df.sort_values(by='year', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Example usage
base_path = './Data/NSF-Downloads-8july2024/'
output_path = './temp'
min_years = 1999  # Replace with your minimum year threshold

df = process_zip_files(base_path, output_path, min_years)
df


,file,year,errors,xmlfiles
0,Data/NSF-Downloads-8july2024/2024.zip,2024,True,8199
1,Data/NSF-Downloads-8july2024/2018.zip,2018,True,12235
2,Data/NSF-Downloads-8july2024/2019.zip,2019,True,13043
3,Data/NSF-Downloads-8july2024/2009.zip,2009,True,15430
4,Data/NSF-Downloads-8july2024/2021.zip,2021,True,12652
5,Data/NSF-Downloads-8july2024/2020.zip,2020,True,13300
6,Data/NSF-Downloads-8july2024/2008.zip,2008,True,12612
7,Data/NSF-Downloads-8july2024/2022.zip,2022,True,11472
8,Data/NSF-Downloads-8july2024/2023.zip,2023,True,11130
9,Data/NSF-Downloads-8july2024/2012.zip,2012,True,12165


In [3]:
df.sort_values(by='year', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,file,year,errors,xmlfiles
0,Data/NSF-Downloads-8july2024/2024.zip,2024,True,8199
1,Data/NSF-Downloads-8july2024/2023.zip,2023,True,11130
2,Data/NSF-Downloads-8july2024/2022.zip,2022,True,11472
3,Data/NSF-Downloads-8july2024/2021.zip,2021,True,12652
4,Data/NSF-Downloads-8july2024/2020.zip,2020,True,13300
5,Data/NSF-Downloads-8july2024/2019.zip,2019,True,13043
6,Data/NSF-Downloads-8july2024/2018.zip,2018,True,12235
7,Data/NSF-Downloads-8july2024/2017.zip,2017,True,12289
8,Data/NSF-Downloads-8july2024/2016.zip,2016,True,12594
9,Data/NSF-Downloads-8july2024/2015.zip,2015,True,13072


In [4]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from datetime import datetime
import pandas as pd
def parse_xml_old(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        data = {
            'AwardID': root.findtext('.//AwardID'),
            'AwardTitle': root.findtext('.//AwardTitle'),
            'Agency': root.findtext('.//AGENCY'),
            'AwardEffectiveDate': root.findtext('.//AwardEffectiveDate'),
            'AwardExpirationDate': root.findtext('.//AwardExpirationDate'),
            'AwardTotalIntnAmount': root.findtext('.//AwardTotalIntnAmount'),
            'AwardAmount': root.findtext('.//AwardAmount'),
            'AwardInstrument': root.findtext('.//AwardInstrument/Value'),
            'Organization_Code': root.findtext('.//Organization/Code'),
            'Directorate_Abbreviation': root.findtext('.//Directorate/Abbreviation'),
            'Directorate_LongName': root.findtext('.//Directorate/LongName'),
            'Division_Abbreviation': root.findtext('.//Division/Abbreviation'),
            'Division_LongName': root.findtext('.//Division/LongName'),
            'ProgramOfficer_Name': root.findtext('.//ProgramOfficer/SignBlockName'),
            'ProgramOfficer_Email': root.findtext('.//ProgramOfficer/PO_EMAI'),
            'ProgramOfficer_Phone': root.findtext('.//ProgramOfficer/PO_PHON'),
            'AbstractNarration': root.findtext('.//AbstractNarration'),
            'MinAmdLetterDate': root.findtext('.//MinAmdLetterDate'),
            'MaxAmdLetterDate': root.findtext('.//MaxAmdLetterDate'),
            'ARRAAmount': root.findtext('.//ARRAAmount'),
            'TRAN_TYPE': root.findtext('.//TRAN_TYPE'),
            'CFDA_NUM': root.findtext('.//CFDA_NUM'),
            'NSF_PAR_USE_FLAG': root.findtext('.//NSF_PAR_USE_FLAG'),
            'FUND_AGCY_CODE': root.findtext('.//FUND_AGCY_CODE'),
            'AWDG_AGCY_CODE': root.findtext('.//AWDG_AGCY_CODE'),
            'Investigator1_FirstName': root.findtext('.//Investigator[1]/FirstName'),
            'Investigator1_LastName': root.findtext('.//Investigator[1]/LastName'),
            'Investigator1_MiddleInitial': root.findtext('.//Investigator[1]/PI_MID_INIT'),
            'Investigator1_Suffix': root.findtext('.//Investigator[1]/PI_SUFX_NAME'),
            'Investigator1_FullName': root.findtext('.//Investigator[1]/PI_FULL_NAME'),
            'Investigator1_Email': root.findtext('.//Investigator[1]/EmailAddress'),
            'Investigator1_NSFID': root.findtext('.//Investigator[1]/NSF_ID'),
            'Investigator1_StartDate': root.findtext('.//Investigator[1]/StartDate'),
            'Investigator1_EndDate': root.findtext('.//Investigator[1]/EndDate'),
            'Investigator1_RoleCode': root.findtext('.//Investigator[1]/RoleCode'),
            'Investigator2_FirstName': root.findtext('.//Investigator[2]/FirstName'),
            'Investigator2_LastName': root.findtext('.//Investigator[2]/LastName'),
            'Investigator2_MiddleInitial': root.findtext('.//Investigator[2]/PI_MID_INIT'),
            'Investigator2_Suffix': root.findtext('.//Investigator[2]/PI_SUFX_NAME'),
            'Investigator2_FullName': root.findtext('.//Investigator[2]/PI_FULL_NAME'),
            'Investigator2_Email': root.findtext('.//Investigator[2]/EmailAddress'),
            'Investigator2_NSFID': root.findtext('.//Investigator[2]/NSF_ID'),
            'Investigator2_StartDate': root.findtext('.//Investigator[2]/StartDate'),
            'Investigator2_EndDate': root.findtext('.//Investigator[2]/EndDate'),
            'Investigator2_RoleCode': root.findtext('.//Investigator[2]/RoleCode'),
            'Investigator3_FirstName': root.findtext('.//Investigator[3]/FirstName'),
            'Investigator3_LastName': root.findtext('.//Investigator[3]/LastName'),
            'Investigator3_MiddleInitial': root.findtext('.//Investigator[3]/PI_MID_INIT'),
            'Investigator3_Suffix': root.findtext('.//Investigator[3]/PI_SUFX_NAME'),
            'Investigator3_FullName': root.findtext('.//Investigator[3]/PI_FULL_NAME'),
            'Investigator3_Email': root.findtext('.//Investigator[3]/EmailAddress'),
            'Investigator3_NSFID': root.findtext('.//Investigator[3]/NSF_ID'),
            'Investigator3_StartDate': root.findtext('.//Investigator[3]/StartDate'),
            'Investigator3_EndDate': root.findtext('.//Investigator[3]/EndDate'),
            'Investigator3_RoleCode': root.findtext('.//Investigator[3]/RoleCode'),
            'Institution_Name': root.findtext('.//Institution/Name'),
            'Institution_City': root.findtext('.//Institution/CityName'),
            'Institution_State': root.findtext('.//Institution/StateName'),
            'Institution_Zip': root.findtext('.//Institution/ZipCode'),
            'Institution_Country': root.findtext('.//Institution/CountryName'),
            'Institution_Phone': root.findtext('.//Institution/PhoneNumber'),
            'Institution_StreetAddress1': root.findtext('.//Institution/StreetAddress'),
            'Institution_StreetAddress2': root.findtext('.//Institution/StreetAddress2'),
            'Institution_CongressDistrict': root.findtext('.//Institution/CONGRESSDISTRICT'),
            'Institution_CongressDistrictOrg': root.findtext('.//Institution/CONGRESS_DISTRICT_ORG'),
            'Institution_OrgUEINum': root.findtext('.//Institution/ORG_UEI_NUM'),
            'Institution_OrgLglBusName': root.findtext('.//Institution/ORG_LGL_BUS_NAME'),
            'Performance_Institution_Name': root.findtext('.//Performance_Institution/Name'),
            'Performance_Institution_City': root.findtext('.//Performance_Institution/CityName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_Zip': root.findtext('.//Performance_Institution/ZipCode'),
            'Performance_Institution_StreetAddress': root.findtext('.//Performance_Institution/StreetAddress'),
            'Performance_Institution_CountryCode': root.findtext('.//Performance_Institution/CountryCode'),
            'Performance_Institution_Country': root.findtext('.//Performance_Institution/CountryName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_CountryFlag': root.findtext('.//Performance_Institution/CountryFlag'),
            'Performance_Institution_CongressDistrict': root.findtext('.//Performance_Institution/CONGRESSDISTRICT'),
            'Performance_Institution_CongressDistrictPerf': root.findtext('.//Performance_Institution/CONGRESS_DISTRICT_PERF'),
            'ProgramElement1_Code': root.findtext('.//ProgramElement[1]/Code'),
            'ProgramElement1_Text': root.findtext('.//ProgramElement[1]/Text'),
            'ProgramElement2_Code': root.findtext('.//ProgramElement[2]/Code'),
            'ProgramElement2_Text': root.findtext('.//ProgramElement[2]/Text'),
            'ProgramElement3_Code': root.findtext('.//ProgramElement[3]/Code'),
            'ProgramElement3_Text': root.findtext('.//ProgramElement[3]/Text'),
            'ProgramReference_Code': root.findtext('.//ProgramReference/Code'),
            'ProgramReference_Text': root.findtext('.//ProgramReference/Text'),
            'Appropriation_Code': root.findtext('.//Appropriation/Code'),
            'Appropriation_Name': root.findtext('.//Appropriation/Name'),
            'Appropriation_SymbolID': root.findtext('.//Appropriation/APP_SYMB_ID'),
            'Fund_Code': root.findtext('.//Fund/Code'),
            'Fund_Name': root.findtext('.//Fund/Name'),
            'Fund_SymbolID': root.findtext('.//Fund/FUND_SYMB_ID'),
            'Fund_Obligation': root.findtext('.//FUND_OBLG')
        }
        return data
    except ET.ParseError:
        print(f'Error parsing file: {file_path}')
        return None


In [23]:
import pandas as pd
from pathlib import Path
from datetime import datetime

def collect_data_from_xml_files(root_folder, year):
    data_list = []
    error_files = []
    yr_folder = root_folder / str(year)
    xmlfiles = list(yr_folder.rglob("*.xml"))
    
    for filename in xmlfiles:
        # Assuming parse_xml is a user-defined function to parse XML files
        data = parse_xml(filename)  # Use filename directly as it contains the full path
        
        if data is not None:
            data['Year'] = year
            data_list.append(data)
        else:
            error_files.append(filename)
    
    return pd.DataFrame(data_list), error_files

# Define the root folder and output directories
root_folder = Path('./temp')
output_csv = Path('./outputcsv')
output_pkl = Path('./outputpkl')

# Ensure output directories exist
output_csv.mkdir(parents=True, exist_ok=True)
output_pkl.mkdir(parents=True, exist_ok=True)

# Add new columns to the DataFrame
df['df_rows'] = 0
df['xml_errors'] = 0
df['xml_error_list'] = None
df['time_parsed'] = None

# Iterate through the DataFrame and process XML files
for index, row in df.iterrows():
    print(f"Processing year: {row['year']} with {row['xmlfiles']} XML files...")
    
    # Collect data and errors from XML files
    data, error_files = collect_data_from_xml_files(root_folder, row['year'])
    
    # Update the DataFrame with new information
    df.loc[index, 'df_rows'] = len(data)
    df.loc[index, 'xml_errors'] = len(error_files)
    df.loc[index, 'xml_error_list'] = str(error_files)
    df.loc[index, 'time_parsed'] = datetime.now()
    
    # Save the processed data
    if not data.empty:
        data.to_csv(output_csv / f'{row["year"]}.csv', index=False)
        data.to_pickle(output_pkl / f'{row["year"]}.pkl')

# Output the final DataFrame
df


Processing year: 2024 with 8199 XML files...
Processing year: 2023 with 11130 XML files...
Processing year: 2022 with 11472 XML files...
Error parsing file: temp/2022/49100422C0020.xml
Processing year: 2021 with 12652 XML files...
Processing year: 2020 with 13300 XML files...
Processing year: 2019 with 13043 XML files...
Processing year: 2018 with 12235 XML files...
Processing year: 2017 with 12289 XML files...
Processing year: 2016 with 12594 XML files...
Processing year: 2015 with 13072 XML files...
Processing year: 2014 with 12109 XML files...


KeyboardInterrupt: 

In [6]:
df.to_csv('0_processing.csv', index=False)

In [12]:
#iterate over the df and read the pickle files to a single dataframe
all=pd.DataFrame()
expected_rows=0
for index, row in df.iterrows():
    print(f"Processing year: {row['year']} with {row['df_rows']} XML files...")
    tmp=pd.read_pickle(output_pkl / f'{row["year"]}.pkl')
    all=pd.concat([all, tmp])
    expected_rows+=row['df_rows']
all.to_csv(output_csv/'all.csv', index=False)  
all.to_pickle(output_pkl/'all.pkl')   
print(f'Expected rows={expected_rows}, actual rows={len(all)}')
all

Processing year: 2024 with 8199 XML files...
Processing year: 2023 with 11130 XML files...
Processing year: 2022 with 11471 XML files...
Processing year: 2021 with 12652 XML files...
Processing year: 2020 with 13300 XML files...
Processing year: 2019 with 13043 XML files...
Processing year: 2018 with 12235 XML files...
Processing year: 2017 with 12289 XML files...
Processing year: 2016 with 12594 XML files...
Processing year: 2015 with 13072 XML files...
Processing year: 2014 with 12109 XML files...
Processing year: 2013 with 11502 XML files...
Processing year: 2012 with 12164 XML files...
Processing year: 2011 with 11648 XML files...
Processing year: 2010 with 13089 XML files...
Processing year: 2009 with 15429 XML files...
Processing year: 2008 with 12603 XML files...
Processing year: 2007 with 12222 XML files...
Processing year: 2006 with 11062 XML files...
Processing year: 2005 with 10728 XML files...
Processing year: 2004 with 10719 XML files...
Processing year: 2003 with 11555 XM

,AwardID,AwardTitle,Agency,AwardEffectiveDate,AwardExpirationDate,AwardTotalIntnAmount,AwardAmount,AwardInstrument,Organization_Code,Directorate_Abbreviation,Directorate_LongName,Division_Abbreviation,Division_LongName,ProgramOfficer_Name,ProgramOfficer_Email,ProgramOfficer_Phone,AbstractNarration,MinAmdLetterDate,MaxAmdLetterDate,ARRAAmount,TRAN_TYPE,CFDA_NUM,NSF_PAR_USE_FLAG,FUND_AGCY_CODE,AWDG_AGCY_CODE,Investigator1_FirstName,Investigator1_LastName,Investigator1_MiddleInitial,Investigator1_Suffix,Investigator1_FullName,Investigator1_Email,Investigator1_NSFID,Investigator1_StartDate,Investigator1_EndDate,Investigator1_RoleCode,Investigator2_FirstName,Investigator2_LastName,Investigator2_MiddleInitial,Investigator2_Suffix,Investigator2_FullName,Investigator2_Email,Investigator2_NSFID,Investigator2_StartDate,Investigator2_EndDate,Investigator2_RoleCode,Investigator3_FirstName,Investigator3_LastName,Investigator3_MiddleInitial,Investigator3_Suffix,Investigator3_FullName,Investigator3_Email,Investigator3_NSFID,Investigator3_StartDate,Investigator3_EndDate,Investigator3_RoleCode,Institution_Name,Institution_City,Institution_State,Institution_Zip,Institution_Country,Institution_Phone,Institution_StreetAddress1,Institution_StreetAddress2,Institution_CongressDistrict,Institution_CongressDistrictOrg,Institution_OrgUEINum,Institution_OrgLglBusName,Performance_Institution_Name,Performance_Institution_City,Performance_Institution_State,Performance_Institution_Zip,Performance_Institution_StreetAddress,Performance_Institution_CountryCode,Performance_Institution_Country,Performance_Institution_CountryFlag,Performance_Institution_CongressDistrict,Performance_Institution_CongressDistrictPerf,ProgramElement1_Code,ProgramElement1_Text,ProgramElement2_Code,ProgramElement2_Text,ProgramElement3_Code,ProgramElement3_Text,ProgramReference_Code,ProgramReference_Text,Appropriation_Code,Appropriation_Name,Appropriation_SymbolID,Fund_Code,Fund_Name,Fund_SymbolID,Fund_Obligation,Year
0,2428698,AGS-PRF: Understanding Compound Heat Wave-Wild...,NSF,12/01/2024,11/30/2026,202000.00,202000,Fellowship Award,06020000,GEO,Directorate For Geosciences,AGS,Div Atmospheric & Geospace Sciences,Yolande L. Serra,yserra@nsf.gov,7032922651,Heat waves when coupled with wildfires are som...,11/21/2024,11/21/2024,,Grant,47.050,1,4900,4900,Raymond,Sukhdeo,,,Raymond Sukhdeo,,000995810,11/21/2024,,Principal Investigator,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"Sukhdeo, Raymond",Davis,California,95616,United States,,,,04,CA04,,,University of California Los Angeles (UCLA),Los Angeles,California,900242593,,US,United States,1,36,CA36,713700,Postdoctoral Fellowships,None,None,None,None,5740,CLIMATE & LARGE-SCALE DYNAMICS,,,,01002526DB,NSF RESEARCH & RELATED ACTIVIT,040100,2025~202000,2024
1,2400114,Collaborative Research: Conference: Brazos Ana...,NSF,04/01/2024,03/31/2027,17250.00,17250,Standard Grant,03040000,MPS,Direct For Mathematical & Physical Scien,DMS,Division Of Mathematical Sciences,Wing Suet Li,winli@nsf.gov,7032924630,This award provides three years of funding to ...,03/25/2024,03/25/2024,,Grant,47.049,1,4900,4900,Jose,Carrion Muniz,,,Jose Carrion Muniz,j.carrion@tcu.edu,000705551,03/25/2024,,Principal Investigator,Travis,Russell,B,,Travis B Russell,travis.b.russell@tcu.edu,000957221,03/25/2024,,Co-Principal Investigator,None,None,None,None,None,None,None,None,None,None,Texas Christian University,FORT WORTH,Texas,761290001,United States,8172577516,3101 BELLAIRE DRIVE NORTH,SUITE 3200,12,TX12,MJCLFGKGULP5,TEXAS CHRISTIAN UNIVERSITY,Texas Christian University,FORT WORTH,Texas,761290001,3101 BELLAIRE DRIVE NORTH,US,United States,1,12,TX12,128100,ANALYSIS PROGRAM,None,None,None,None,7556,CONFERENCE AND WORKSHOPS,,,,01002425DB,NSF RESEARCH & RELATED ACTIVIT,040100,2024~17250,2024
2,2418983,Collaborative Research: Incubation Project: A ...,NSF,10/01/2024,09/30/2025,4366.00,4366,Standard Grant,04010000,SBE,"Direct F

In [21]:
data=pd.read_pickle(output_pkl/'all.pkl')

In [13]:
replace_name_university = {'Stony Brook University': 'SUNY at Stony Brook',
 'The University of Vermont': 'University of Vermont & State Agricultural College',
 'University of North Carolina-Charlotte': 'University of North Carolina at Charlotte',
 'Pennsylvania State University': 'Pennsylvania State Univ University Park',
 'The University of Akron': 'University of Akron',
 'Rice University': 'William Marsh Rice University',
 'The University of New Mexico': 'University of New Mexico',
 'San Diego State University': 'San Diego State University Foundation', # corrigida San Diego State University
 'Carnegie Mellon University': 'Carnegie-Mellon University',
 'Oklahoma State University': 'Oklahoma State University Education and Research Foundation Inc',#corrigida Oklahoma State University',
 'University of Colorado Boulder': 'University of Colorado at Boulder',# corrigida
 'University of Idaho': 'Idaho State University',#nao tem grants do ecosistema I-corps no arquivo da nsf 
 'University of Nevada, Las Vegas':'University of Nevada Las Vegas',
 'UC Santa Cruz': 'University of California-Santa Cruz',
 'University of Michigan': 'Regents of the University of Michigan - Ann Arbor',
 'Hampton University': 'Hampton University', # nao tem grants do ecosistema I-corps no arquivo da nsf 
 'The University of Toledo': 'University of Toledo',
 'Georgia Tech': 'Georgia Tech Research Corporation - GA Tech Research Institute',#corrigida Georgia Tech Research Corporation',
 'University of Maryland College Park': 'University of Maryland, College Park',
 'Icahn School of Medicine at Mount Sinai': 'Mount Sinai School of Medicine',#corrigida possui apenas I-Corps Icahn School of Medicine at Mount Sinai', 
 'North Dakota State University': 'North Dakota State University Fargo',
 'University of California San Diego': 'University of California-San Diego',
 'University at Albany': 'SUNY at Albany', #University at Albany',
 'Tennessee State University': 'Tennessee State University',#East Tennessee State University recebeu um I-Corps, porem a Tennessee State University nao recebeu nada 
 'The City University of New York': 'CUNY City University of New York',#'CUNY Graduate School University Center'
 'Virginia Tech': 'Virginia Polytechnic Institute and State University',
 'UC Riverside': 'University of California-Riverside',
 'UC San Francisco': 'University of California-San Francisco',
 'UC Berkeley': 'University of California-Berkeley',
 'Cornell University': 'Cornell Univ - State: AWDS MADE PRIOR MAY 2010',
 'The University of Texas at San Antonio': 'University of Texas at San Antonio',
 'The University of Alabama':'University of Alabama Tuscaloosa',# 'University of Alabama at Birmingham',
 'University of Miami' : 'University of Miami',# nao tem grants do ecosistema I-corps no arquivo da nsf 
 'Binghamton University': 'SUNY at Binghamton',
 'The University of Texas at El Paso': 'University of Texas at El Paso',
 'UC Davis': 'University of California-Davis',
 'UCLA': 'University of California-Los Angeles',#corrigido
 'University of Wisconsin–Milwaukee': 'University of Wisconsin-Milwaukee',
 'South Dakota School of Mines & Technology': 'South Dakota School of Mines and Technology',
 'Rutgers University': 'Rutgers University New Brunswick',
 'UC Irvine': 'University of California-Irvine',
 'University of North Dakota': 'University of North Dakota Main Campus',
 'University of Hawai‘i': 'University of Hawaii',
 'University of Minnesota Twin Cities': 'University of Minnesota-Twin Cities',
 'University of Massachusetts Medical School': 'University of Massachusetts Medical School',#nao tem grants do ecosistema I-corps no arquivo da nsf
 'University at Buffalo': 'SUNY at Buffalo',#corrigida University at Buffalo - STOR Technology Incubator',
 'Morehouse College': 'Morehouse College',#nao tem grants do ecosistema I-corps no arquivo da nsf
 'The University of Texas at Austin': 'University of Texas at Austin',#corrigida
 'Dakota State University': 'South Dakota State University',
 'University of Illinois Urbana-Champaign': 'University of Illinois at Urbana-Champaign',
 'USC': 'University of Southern California',#corrigida USC Department of Earth Sciences',
 'University of Nebraska Omaha': 'University of Nebraska at Omaha',# nao tem grants do ecosistema I-corps no arquivo da nsf 
 'The George Washington University': 'George Washington University',
 'Rochester Institute of Technology': 'Rochester Institute of Tech',
 'The University of Iowa': 'University of Iowa',
 'University of Alaska Fairbanks': 'University of Alaska Fairbanks Campus',
 'University of South Dakota': 'South Dakota State University',
 'West Virginia University': 'West Virginia University Research Corporation', # 'West Virginia University', apenas  'West Virginia University Research Corporation' recebeu I-Corps 
 'Meharry Medical College': 'Meharry Medical College',#nao tem grants do ecosistema I-corps no arquivo da nsf
 'University of North Carolina-Chapel Hill': 'University of North Carolina at Chapel Hill',
 'University of Virginia': 'University of Virginia Main Campus',# corrigida'University of Virginia',
 'The University of Utah': 'University of Utah',
 'Missouri University of Science & Technology': 'Missouri University of Science and Technology',
 'University of Central Florida':'The University of Central Florida Board of Trustees',#corrigida 'University of Central Florida',
 'UC Santa Barbara': 'University of California-Santa Barbara',
 'University of Tennessee, Knoxville': 'University of Tennessee Knoxville', #acrescentada}
 'UC Berkeley':'Lawrence Berkeley Laboratory'}

In [14]:
# Define a dictionary with universities and their corresponding hub names for efficient lookup
hub_members = {
    "Desert and Pacific": [
        "Arizona State University", "Boise State University", "Northern Arizona University",
        "San Diego State University", "University of Arizona", "University of California San Diego",
        "University of Hawai‘i", "University of Idaho", "University of Nevada, Las Vegas"
    ],
    "Great Lakes": [
        "University of Michigan", "Iowa State University", "Michigan Technological University",
        "Missouri University of Science & Technology", "Purdue University", "The University of Akron",
        "University of Chicago", "University of Illinois Urbana-Champaign", "The University of Iowa",
        "University of Minnesota Twin Cities", "University of Nebraska-Lincoln", "The University of Toledo",
        "University of Wisconsin-Madison", "University of Wisconsin–Milwaukee"
    ],
    "Great Plains": [
        "North Dakota State University", "Dakota State University", "South Dakota School of Mines & Technology",
        "South Dakota State University", "University of Montana", "University of Nebraska Omaha",
        "University of North Dakota", "University of South Dakota", "University of Wyoming"
    ],
    "Interior Northeast": [
        "Cornell University", "Binghamton University", "Dartmouth College",
        "Rochester Institute of Technology", "Syracuse University", "University at Buffalo",
        "University of Pittsburgh", "University of Rochester", "The University of Vermont", "West Virginia University"
    ],
    "Mid-Atlantic": [
        "University of Maryland College Park", "Carnegie Mellon University", "The George Washington University",
        "Hampton University", "Howard University", "Johns Hopkins University",
        "North Carolina State University", "Pennsylvania State University", "University of North Carolina-Chapel Hill",
        "University of North Carolina-Charlotte", "University of Pennsylvania", "Virginia Tech"
    ],
    "Mid-South": [
        "Vanderbilt University", "George Mason University", "Jackson State University",
        "Meharry Medical College", "Tennessee State University", "University of Kentucky",
        "University of Louisville", "University of Tennessee, Knoxville", "University of Virginia"
    ],
    "New England": [
        "Massachusetts Institute of Technology", "Brown University", "Harvard University",
        "Northeastern University", "Tufts University", "University of Maine",
        "University of Massachusetts Amherst", "University of New Hampshire"
    ],
    "New York": [
        "The City University of New York", "Clarkson University", "Columbia University",
        "Icahn School of Medicine at Mount Sinai", "New York University", "Rensselaer Polytechnic Institute",
        "Rockefeller University", "Stevens Institute of Technology", "Stony Brook University",
        "University at Albany", "University of Massachusetts Medical School"
    ],
    "Northeast": [
        "Princeton University", "Delaware State University", "Drexel University",
        "Lehigh University", "New Jersey Institute of Technology", "Rutgers University",
        "Rowan University", "Temple University", "University of Connecticut",
        "University of Delaware", "Yale University"
    ],
    "Northwest": [
        "UC Berkeley", "Oregon State University", "University of Alaska Fairbanks",
        "UC Davis", "UC Irvine", "UC San Francisco", "UC Santa Cruz", "University of Washington"
    ],
    "Southeast": [
        "Georgia Tech", "Clemson University", "Morehouse College",
        "The University of Alabama", "University of Central Florida", "University of Florida",
        "University of Miami", "University of South Florida"
    ],
    "Southwest": [
        "The University of Texas at Austin", "Louisiana State University", "Mississippi State University",
        "New Mexico State University", "Oklahoma State University", "Rice University",
        "Texas A&M University", "The University of Texas at El Paso", "The University of Texas at San Antonio"
    ],
    "West": [
        "USC", "California Institute of Technology", "Colorado School of Mines",
        "Colorado State University", "UCLA", "UC Riverside", "UC Santa Barbara",
        "University of Colorado Boulder", "The University of New Mexico", "The University of Utah"
    ]
}

In [15]:
NODES_members = {'Massachusetts Institute of Technology': ['University of New England'],
 'National Collegiate Inventors and Innovators Alliance': ['National Collegiate Inventors and Innovators Alliance'],
 'University of California-Los Angeles': ['University of California-Los Angeles'],
 'Cleveland Clinic Foundation': ['Cleveland Clinic Foundation'],
 'Case Western Reserve University': ['Regents of the University of Michigan - Ann Arbor',
                                     'Ohio State University',
                                     'University of Cincinnati Main Campus',
                                     'Northwestern University',
                                     "Children's Hospital Medical Center"],
 'University of Southern California': ['University of Southern California'],
 'Howard University': ['Hampton University',
                       'LELAND JUNIOR STANFORD UNIVERSITY',
                       'National Collegiate Inventors and Innovators Alliance'],
 'University of California-Riverside': ['University of California-Berkeley',
                                        'University of California-San Francisco',
                                        'LELAND JUNIOR STANFORD UNIVERSITY',
                                        'University of California-Davis',
                                        'University of California-Irvine',
                                        'University of California - Merced',
                                        'University of California-Riverside',
                                        'University of California-San Diego',
                                        'University of California-Santa Barbara',
                                        'University of California-Santa Cruz'],
 'Regents of the University of Michigan - Ann Arbor': ['University of Illinois at Urbana-Champaign',
                                                       'Purdue University'],
 'Purdue University': ['University of Rochester',
                       'Rochester Institute of Tech']}

In [17]:

# replace names of university 
for  hub in hub_members.keys():
    for  i, university in  enumerate(hub_members[hub]):
        if university in replace_name_university:
            hub_members[hub][i] = replace_name_university[university]

# Perform string manipulation to normalize both the unmatched list and data for better matching
def normalize_string(s):
    return s.strip().lower().replace("–", "-").replace("‘", "'").replace("’", "'")

# Normalize hub members for lookup
university_to_hub = {
    normalize_string(university): hub
    for hub, universities in hub_members.items()
    for university in universities
}

# Normalize institution names in the dataset for matching
all['Normalized_Institution_Name'] = all['Institution_Name'].apply(
    lambda x: normalize_string(x) if isinstance(x, str) else ''
)

# Populate the HUB column based on the normalized names
all['HUB'] = all['Normalized_Institution_Name'].apply(
    lambda x: university_to_hub.get(x, "NOHUB") if x else np.nan
)

In [22]:
#filter data where investicator Investigator3_NSFID is not null
data.loc[data['Investigator3_NSFID'].notnull()]


,AwardID,AwardTitle,Agency,AwardEffectiveDate,AwardExpirationDate,AwardTotalIntnAmount,AwardAmount,AwardInstrument,Organization_Code,Directorate_Abbreviation,Directorate_LongName,Division_Abbreviation,Division_LongName,ProgramOfficer_Name,ProgramOfficer_Email,ProgramOfficer_Phone,AbstractNarration,MinAmdLetterDate,MaxAmdLetterDate,ARRAAmount,TRAN_TYPE,CFDA_NUM,NSF_PAR_USE_FLAG,FUND_AGCY_CODE,AWDG_AGCY_CODE,Investigator1_FirstName,Investigator1_LastName,Investigator1_MiddleInitial,Investigator1_Suffix,Investigator1_FullName,Investigator1_Email,Investigator1_NSFID,Investigator1_StartDate,Investigator1_EndDate,Investigator1_RoleCode,Investigator2_FirstName,Investigator2_LastName,Investigator2_MiddleInitial,Investigator2_Suffix,Investigator2_FullName,Investigator2_Email,Investigator2_NSFID,Investigator2_StartDate,Investigator2_EndDate,Investigator2_RoleCode,Investigator3_FirstName,Investigator3_LastName,Investigator3_MiddleInitial,Investigator3_Suffix,Investigator3_FullName,Investigator3_Email,Investigator3_NSFID,Investigator3_StartDate,Investigator3_EndDate,Investigator3_RoleCode,Institution_Name,Institution_City,Institution_State,Institution_Zip,Institution_Country,Institution_Phone,Institution_StreetAddress1,Institution_StreetAddress2,Institution_CongressDistrict,Institution_CongressDistrictOrg,Institution_OrgUEINum,Institution_OrgLglBusName,Performance_Institution_Name,Performance_Institution_City,Performance_Institution_State,Performance_Institution_Zip,Performance_Institution_StreetAddress,Performance_Institution_CountryCode,Performance_Institution_Country,Performance_Institution_CountryFlag,Performance_Institution_CongressDistrict,Performance_Institution_CongressDistrictPerf,ProgramElement1_Code,ProgramElement1_Text,ProgramElement2_Code,ProgramElement2_Text,ProgramElement3_Code,ProgramElement3_Text,ProgramReference_Code,ProgramReference_Text,Appropriation_Code,Appropriation_Name,Appropriation_SymbolID,Fund_Code,Fund_Name,Fund_SymbolID,Fund_Obligation,Year
5,2413168,IUCRC Phase I George Mason University: Center ...,NSF,08/01/2024,07/31/2029,750000.00,150000,Continuing Grant,05050000,CSE,Direct For Computer & Info Scie & Enginr,CNS,Division Of Computer and Network Systems,Mohan Kumar,mokumar@nsf.gov,7032927408,We are on the verge of the development of the ...,07/11/2024,07/27/2024,,Grant,47.070,1,4900,4900,Vijay,Shah,K,,Vijay K Shah,vkshah2@ncsu.edu,000807960,07/11/2024,07/27/2024,Former Co-Principal Investigator,Duminda,Wijesekera,,,Duminda Wijesekera,dwijesek@gmu.edu,000383359,07/11/2024,,Co-Principal Investigator,Brian,Mark,L,,Brian L Mark,bmark@gmu.edu,000189636,07/11/2024,,Principal Investigator,George Mason University,FAIRFAX,Virginia,220304422,United States,7039932295,4400 UNIVERSITY DR,,11,VA11,EADLFP7Z72E5,GEORGE MASON UNIVERSITY,George Mason University,FAIRFAX,Virginia,220304422,4400 UNIVERSITY DR,US,United States,1,11,VA11,576100,IUCRC-Indust-Univ Coop Res Ctr,None,None,None,None,5761,INDUSTRY/UNIV COOP RES CENTERS,,,,01002425DB,NSF RESEARCH & RELATED ACTIVIT,040100,2024~150000,2024
28,2422389,Advanced Research Training in Physiology - In...,NSF,08/01/2024,07/31/2025,345417.00,345417,Standard Grant,08070000,BIO,Direct For Biological Sciences,MCB,Div Of Molecular and Cellular Bioscience,Richard Cyr,rcyr@nsf.gov,7032928440,The Advanced Research Training Course in Physi...,07/30/2024,07/30/2024,,Grant,47.074,1,4900,4900,Amy,Gladfelter,S,,Amy S Gladfelter,amy.gladfelter@duke.edu,000213612,07/30/2024,,Co-Principal Investigator,Clifford,Brangwynne,P,,Clifford P Brangwynne,cbrangwy@princeton.edu,000623829,07/30/2024,,Co-Principal Investigator,Linda,Hyman,,,Linda Hyman,lhyman@mbl.edu,000745449,07/30/2024,,Principal Investigator,Marine Biological Laboratory,WOODS HOLE,Massachusetts,025431015,United States,5082897243,7 M B L ST,,09,MA09,M2XKLRTA9G44,THE MARINE BIOLOGICAL LABORATORY,Marine Biological Laboratory,WOODS HOLE,Massachusetts,025431015,7 M B L ST,US,United States,1,09,MA09,550000,NEURAL SYSTEMS CLUSTER,727500,Cross-

In [ ]:
2413168

KeyError: 'Grant'

In [ ]:
# Perform string manipulation to normalize both the unmatched list and data for better matching
def normalize_string(s):
    return s.strip().lower().replace("–", "-").replace("‘", "'").replace("’", "'")
